In [1]:
source("/home/mount/project/config_sing.R")

── Attaching packages ──────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ─────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




In [70]:
CHROM = "chr17"

In [71]:
###################################################
# Import library size
###################################################

### Helper function to get
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext(basename(Fpath))) %>%
    mutate(Group = get_sample(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)
head(dat_lib)

Size,Sample,Group
<dbl>,<chr>,<chr>
18666630,Input1,Input
20167924,Input2,Input
23280988,Input3,Input
19003938,Input4,Input
15325016,Input5,Input
48376253,TFX2_AZD2906,TFX_AZD2906


In [72]:
###################################################
# Import annotated fragments
###################################################

### set column names and types
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

### set samples
SAMPLES = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

### import bed files for each sample 
fdiry = file.path(FD_RES, "annotation_fragment", "filter_motif_score095")
fname = paste0(CHROM, ".bed", ".gz") 

lst_dat = lapply(SAMPLES, function(sam){
    ### set path
    fpath = file.path(fdiry, sam, fname)
    print(fpath)
    
    ### import data
    dat = read_tsv(fpath, col_types=ctypes, col_names=cnames) %>% mutate(Sample = sam)
    return(dat)
})

### arrange data
dat_ann_frag = bind_rows(lst_dat)
lst_dat = NULL

### check point: print message
head(dat_ann_frag, 3)
print(colnames(dat_ann_frag))

print("Dimension")
print(dim(dat_ann_frag))

print("Size")
print(object.size(dat_ann_frag), units="Mb")

[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input2/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input3/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input4/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input5/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX2_DMSO/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX3_DMSO/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX4_DMSO/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/TFX5_DMSO/chr17.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/a

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Chrom_MTF,Start_MTF,End_MTF,Motif,Score,Overlap,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
chr17,201123,201925,1,chr17,201304,201315,ZFX,10.9102,11,Input1
chr17,201123,201925,1,chr17,201497,201505,FOX/1,11.1330,8,Input1
chr17,201123,201925,1,chr17,201580,201588,FOX/1,11.1330,8,Input1


 [1] "Chrom_Frag" "Start_Frag" "End_Frag"   "Count_Frag" "Chrom_MTF" 
 [6] "Start_MTF"  "End_MTF"    "Motif"      "Score"      "Overlap"   
[11] "Sample"    
[1] "Dimension"
[1] 113443263        11
[1] "Size"
9520.6 Mb


In [73]:
###################################################
# Preprocess
###################################################

### grouped by motif cluster and 
### split the annotated fragments into list
dat    = dat_ann_frag
lst    = dat %>% group_by(Motif) %>% group_split
motifs = lapply(lst, function(dat){unique(dat$Motif)}) %>% unlist
names(lst) = motifs

### get the list
lst_frag = lapply(lst, function(dat){
    tmp = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Motif, Sample) %>%
        distinct()
    return(tmp)
})

In [66]:
head(lst_frag[[2]])

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chrY,11096035,11097300,1,AP1/1,Input1
chrY,11096699,11097734,1,AP1/1,Input1
chrY,11096727,11097722,1,AP1/1,Input1
chrY,11102283,11103258,1,AP1/1,Input1
chrY,56859582,56860521,1,AP1/1,Input1
chrY,56883167,56884086,1,AP1/1,Input1


In [67]:
length(lst_frag)

[1] 217

In [74]:
### filter out motifs that have almost no fragment in total
cat("+++++ Preprocess: filter motifs +++++\n")
cat("Before filteration: #Motifs =", length(lst_frag), "\n")

THRESHOLD = 10
lst = lst_frag
cnt = lapply(lst, function(dat){sum(dat$Count_Frag)})
lst = lst[cnt > 10]
lst_frag = lst

cat("Threshold =", THRESHOLD, "\n")
cat("After filteration: #Motifs =", length(lst_frag), "\n")

+++++ Preprocess: filter motifs +++++
Before filteration: #Motifs = 242 
Threshold = 10 
After filteration: #Motifs = 242 


In [78]:
tmp = unlist(cnt)
summary(tmp)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
     93   41060  157206  482728  571668 7243033 

In [80]:
sum(cnt < 100)

[1] 1

In [68]:
tmp = head(lst_frag)
lapply(tmp, function(dat){sum(dat$Count_Frag)})

$AIRE
[1] 1

$`AP1/1`
[1] 228

$`AP1/2`
[1] 477

$BATF
[1] 10

$`BCL6/1`
[1] 1

$`BCL6/2`
[1] 17

In [60]:
lapply(tmp, nrow)

$`AP1/1`
[1] 216

$`AP1/2`
[1] 422

$`BCL6/2`
[1] 17

In [64]:
length(tmp)

[1] 173

In [9]:
lapply(head(lst_frag), head)

Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chrY,11428494,11429537,1,AIRE,TFX2_DMSO
Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chrY,11096035,11097300,1,AP1/1,Input1
chrY,11096699,11097734,1,AP1/1,Input1
chrY,11096727,11097722,1,AP1/1,Input1
chrY,11102283,11103258,1,AP1/1,Input1
chrY,56859582,56860521,1,AP1/1,Input1
chrY,56883167,56884086,1,AP1/1,Input1


In [11]:
dat = lst_frag[[1]]
print(dim(dat))
head(dat)

[1] 1 6


Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chrY,11428494,11429537,1,AIRE,TFX2_DMSO


In [ ]:
lapply()

In [12]:
dat = lst_frag[[2]]
print(dim(dat))
head(dat)

[1] 216   6


Chrom_Frag,Start_Frag,End_Frag,Count_Frag,Motif,Sample
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
chrY,11096035,11097300,1,AP1/1,Input1
chrY,11096699,11097734,1,AP1/1,Input1
chrY,11096727,11097722,1,AP1/1,Input1
chrY,11102283,11103258,1,AP1/1,Input1
chrY,56859582,56860521,1,AP1/1,Input1
chrY,56883167,56884086,1,AP1/1,Input1


In [46]:
    dat = lst_frag[[1]]

    ### get fragments
    dat = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample) %>%
        distinct()

    ### count
    dat = dat %>% group_by(Sample) %>% summarise(Value = sum(Count_Frag))

    ### normalize counts by library size
    tmp = dat %>% left_join(dat_lib, by="Sample")
    tmp = tmp %>%
        mutate(Norm_Value    = Value / Size) %>% 
        mutate(Lognorm_Value = log10(Value) - log10(Size)) %>%
        mutate(X = Group)

    idxs  = c("Input", "TFX_DMSO", "TFX_Dex")
    tmp$X = factor(tmp$X, levels=idxs)
    X = model.matrix(~X, tmp)
    X[,"XTFX_DMSO"] = X[,"XTFX_DMSO"] + X[,"XTFX_Dex"]
    y = tmp$Norm_Value

    ### fit model and get the summary
    fit = lm(y ~ X + 0)
    res = summary(fit)

In [47]:
tmp

Sample,Value,Size,Group,Norm_Value,Lognorm_Value,X
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<fct>
TFX2_DMSO,1,43844606,TFX_DMSO,2.280782e-08,-7.641916,TFX_DMSO


In [48]:
X

,(Intercept),XTFX_DMSO,XTFX_Dex
1,1,1,0


In [49]:
y

[1] 2.280782e-08

In [50]:
fit


Call:
lm(formula = y ~ X + 0)

Coefficients:
X(Intercept)    XXTFX_DMSO     XXTFX_Dex  
   2.281e-08            NA            NA  


In [51]:
res


Call:
lm(formula = y ~ X + 0)

Residuals:
ALL 1 residuals are 0: no residual degrees of freedom!

Coefficients: (2 not defined because of singularities)
              Estimate Std. Error t value Pr(>|t|)
X(Intercept) 2.281e-08        NaN     NaN      NaN
XXTFX_DMSO          NA         NA      NA       NA
XXTFX_Dex           NA         NA      NA       NA

Residual standard error: NaN on 0 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:    NaN 
F-statistic:   NaN on 1 and 0 DF,  p-value: NA
